# Examen de cómputo matricial equipo SVD

**Fecha**

19 de Abril de 2020

**Objetivo**

Programar en el lenguaje R el método de eliminación por bloques para la solución de un sistema de ecuaciones $$Ax=b$$ mediante el método de descomposición en valores singulares (DVS o SVD por siglas en inglés), a través del Algoritmo One-Sided Jacobi.

**Especificaciones de ambiente común de trabajo**

Para trabajar y a efecto de tener un entorno común de trabajo para el desarrollo del proyecto, por favor emplear la imagen de docker basada en R del curso MNO 2020 (palmoreck/jupyterlab_r_kernel:1.1.0)

```
docker run --rm -v `pwd`:/datos --name jupyterlab_r_kernel_local -p 8888:8888 -d palmoreck/jupyterlab_r_kernel:1.1.0

```

*Nota:* el comando "-v \`pwd\`:/datos", permite montar el directorio actual en donde el usuario se encuentre situada en terminal como un volumen de la imagen de docker, dentro del directorio "/datos".

**Comentarios**

Para mayor información consultar el [Project Board](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/projects/1), la [especificación simplificada del algoritmo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/References/Simplified_SVD_OneSidedJacobi_Algorithm.md) así como las instrucciones de los issues correspondientes


## 1. Funciones auxiliares

Las siguientes funciones serán procedimientos de apoyo en el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 1.1 Generación de índices

In [5]:
indices <- function(n) {
  # Crea una lista de tamaño (n-1)n/2 con pares de índices de la siguiente
  #  manera: (1,2),..,(1,n),(2,3),..,(2,n),...,(n-1,n)
  # Args: 
  #    n: número entero postivo 
  #       se refiere al número de columnas
  #Returns:
  #    lista con pares de índices
    a <- NULL
    b <- NULL
    indices <- NULL
    for (i in 1:(n-1)){
    a <- append(a,rep(i,n-i))
    b <- append(b,seq(i+1,n))    
    }
    for(i in 1:round(n*(n-1)/2))
    indices[[i]] <- list(c(a[i], b[i]))
    indices


}

### 1.2 Verificación de ortogonalidad entre vectores

In [ ]:
#Función Ortogonalidad
ortogonal <- function(u,v,TOL=10^-8){
  #Función que verifica si dos vectores son ortogonales, arrojando un 1 si lo es, y un 0 si no lo es.
  #Args: u, v como vectores de la misma dimensión.Y un valor real de tolerancia TOL con valor prefijado de 10^-8.
  #Returns: Valor booleano 0 (no son ortongoales), 1 (son ortogonales)
  dot.prod <- u%*%v 
  norm.u <- norm(u,type="2")
  norm.v <- norm(v,type="2")
  orth <- dot.prod / (norm.u * norm.v)
  if (orth< TOL){
    print("1")
  } else {
    print("0")
  }
}

### 1.3 Función signo

In [1]:
signo<-function(x) {
  # Indica el signo de un número
  # Args: 
  #    X (numeric): número a revisar
  #Returns:
  #    1 si el número es positivo o cero
  #    -1 si el número es negativo
  
  ifelse(x<0,-1,1)
  }

### 1.4 Solver dada descomposición SVD

In [ ]:
# Función Solver

solver <- function(U,S,V,b){
    # Función que devuelve la solcuón del sistema de ecuaciones Ax =b.
    # Se utilizó la función backsolve para resolver el sistema triangular.
    # NOTA: al ser S diagonal es indistinto si es traingular inferior o superior.
    # Args: U (mxm),V(nxn), S(mxn) matriz diagonal y b (m) un vector.
    # Returns: Vector x
  d = backsolve(t(U),b)
  x = V%*%d
  return(x)
}

## 2. Algoritmo SVD y solución de sistema lineal

Esta sección aborda el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 2.1 One-sided Jacobi numerical aproximación

In [3]:
svd_jacobi_aprox <- function(A,TOL,maxsweep){
    #Función que devuelve una lista con las matrices U, S, V que aproximan númericamente a A 
    #por medio del método de One-sided Jacobi 
    # Args: 
    #    A: matriz a la que se le calculará su SVD
    #    TOL (numeric): controla la convergencia del método
    #    maxsweep (numeric): número máximo de sweeps 
    #Returns: lista con matrices U, S, V


    #dimensiones
    n<-dim(A)[2] #numero de columnas
    m<-dim(A)[1] #numero de filas
    nmax<-n*(n-1)/2

    #inicialza valores del ciclo
    ak<-A
    vk<-diag(n)
    sig <- NULL
    uk <- ak
    num_col_ortogonal<-0
    k<-0

    while(k<=maxsweep & num_col_ortogonal<nmax){
    num_col_ortogonal<-0
  
    ind <- indices(n)
    for(i in 1:nmax){
      col_j<-ak[,ind[[i]][[1]][2]]
      col_i<-ak[,ind[[i]][[1]][1]]
    
      #comprueba ortogonalidad  
      if(ortogonal(col_i,col_j,TOL==1)){
        num_col_ortogonal<-num_col_ortogonal+1
      }
      else{
        #calcula coeficientes de la matriz
        a<-sum(col_i*col_i)
        b<-sum(col_j*col_j)
        c<-col_i%*%col_j
        
        #calcula la rotacion givens que diagonaliza
        epsilon<-(b-a)/(2*c)
        t<-signo(epsilon)/(abs(epsilon)+sqrt(1+epsilon**2))
        cs<-1/sqrt(1+t**2)
        sn<-cs*t
        
        #actualiza las columnas de la matriz ak
        for(l in seq(1,m)){
          temp<-ak[l,ind[[i]][[1]][1]]
          ak[l,ind[[i]][[1]][1]]<-cs*temp-sn*ak[l,ind[[i]][[1]][2]]
          ak[l,ind[[i]][[1]][2]]<-sn*temp+cs*ak[l,ind[[i]][[1]][2]]
        }
        
        #actualiza las columnas de la matriz vk
        for(l in seq(1,n)){
          temp<-vk[l,ind[[i]][[1]][1]]
          vk[l,ind[[i]][[1]][1]]<-cs*temp-sn*vk[l,ind[[i]][[1]][2]]
          vk[l,ind[[i]][[1]][2]]<-sn*temp+cs*vk[l,ind[[i]][[1]][2]]
        
      }
    }
  }
  k<-k+1
}
        for(i in 1:n){
        sig<- append(sig,norm(ak[,i],type ="2"))
    }


    for(i in 1:n){
        uk[,i] <- ak[,i]/sig[i]
    }

    S <- diag(sig)

    list(S = S, U = uk, V= vk)
 }   

In [2]:
#funciones auxilieres que aun no funcionan bien arriba
signo<-function(x){
  ifelse(x<0,-1,1)
}

ortogonal<-function(col_i,col_j,TOL){
  if(sum(col_i*col_j)/(sqrt(sum(col_i*col_i))*sqrt(sum(col_j*col_j)))<TOL){
    ret<-1
  }
  else{
    ret<-0
  }
  ret
}


In [6]:
#Ejemplo
#parametros de entrada
A<-matrix(c(3,2,2,3,2,-2),nrow=2)
TOL<-10**-8
maxsweep<-20
#Función
svd_jacobi_aprox(A,TOL,maxsweep)

1.570092e-16,0,0
0.000000e+00,5,0
0.000000e+00,0,3
-0.7071068,0.7071068,0.7071068
0.7071068,0.7071068,-0.7071068
0.6666667,0.7071068,0.2357023
-0.6666667,0.7071068,-0.2357023
-0.3333333,0.0000000,0.9428090


### 2.2 Linear solver aproximating SVD decomposition using One-sided Jacobi algorithm